In [1]:
!pip install python-box

In [2]:
import numpy as np
import yaml
from box import Box

import torch
import torch.nn as nn
import torch.optim as optim

import simmim
from swin_v2 import SwinTransformerV2

from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import time

from timm.data import Mixup
import transformers

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
simmim_config = yaml.load(open('config/pretrain.yaml'), Loader=yaml.FullLoader)
simmim_config

{'MODEL': {'TYPE': 'swinv2',
  'NAME': 'simmim_pretrain',
  'DROP_PATH_RATE': 0.1,
  'SIMMIM': {'NORM_TARGET': {'ENABLE': True, 'PATCH_SIZE': 47}},
  'SWIN': {'EMBED_DIM': 96,
   'DEPTHS': [2, 2, 6, 2],
   'NUM_HEADS': [3, 6, 12, 24],
   'WINDOW_SIZE': 7,
   'PATCH_SIZE': 4}},
 'DATA': {'IMG_SIZE': 224,
  'MASK_PATCH_SIZE': 32,
  'MASK_RATIO': 0.6,
  'BATCH_SIZE': 512,
  'NUM_WORKERS': 24,
  'DATA_PATH': '../../data/sports/train'},
 'TRAIN': {'EPOCHS': 100,
  'WARMUP_EPOCHS': 10,
  'BASE_LR': '1e-4',
  'WARMUP_LR': '5e-7',
  'WEIGHT_DECAY': 0.05,
  'CLIP_GRAD': 5,
  'LR_SCHEDULER': {'NAME': 'multistep', 'GAMMA': 0.1, 'MULTISTEPS': [70]}},
 'PRINT_FREQ': 100,
 'SAVE_FREQ': 5,
 'TAG': 'simmim_pretrain__swinv2_tiny__img224_window7__800ep'}

In [4]:
encoder_config = {'img_size':simmim_config['DATA']['IMG_SIZE'], 
                'patch_size':simmim_config['MODEL']['SWIN']['PATCH_SIZE'], 
                'in_chans':3, 
                'num_classes':100,
                'embed_dim':simmim_config['MODEL']['SWIN']['EMBED_DIM'], 
                'depths':simmim_config['MODEL']['SWIN']['DEPTHS'], 
                'num_heads':simmim_config['MODEL']['SWIN']['NUM_HEADS'],           
                'window_size':simmim_config['MODEL']['SWIN']['WINDOW_SIZE'], 
                'mlp_ratio':4., 
                'qkv_bias':True, 
                'qk_scale':None,
                'drop_rate':0., 
                'attn_drop_rate':0., 
                'drop_path_rate':simmim_config['MODEL']['DROP_PATH_RATE'],
                'norm_layer':nn.LayerNorm, 
                'patch_norm':True, 
                'pretrained_window_sizes':[0,0,0,0],
                'ape':True}

encoder_stride = 32
in_chans = encoder_config['in_chans']
patch_size = encoder_config['patch_size']

In [5]:
encoder = simmim.SwinTransformerV2ForSimMIM(**encoder_config)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
model = simmim.SimMIM( encoder=encoder, 
                       encoder_stride=encoder_stride, 
                       in_chans=in_chans, 
                       patch_size=patch_size)

In [7]:
simmim_config = Box(simmim_config)
dataloader = simmim.build_loader_simmim(simmim_config)

samples = next(iter(dataloader))
len(samples)

3

In [8]:
samples[0].shape, samples[1].shape, samples[2].shape 

(torch.Size([512, 3, 224, 224]), torch.Size([512, 56, 56]), torch.Size([512]))

In [9]:
base_lr = float(simmim_config.TRAIN.BASE_LR)
weight_decay = simmim_config.TRAIN.WEIGHT_DECAY
optimizer = optim.AdamW(model.parameters(), lr=base_lr, weight_decay=weight_decay)
warmup_epochs = simmim_config.TRAIN.WARMUP_EPOCHS
train_epochs = simmim_config.TRAIN.EPOCHS

multisteps = simmim_config.TRAIN.LR_SCHEDULER.MULTISTEPS
gamma = simmim_config.TRAIN.LR_SCHEDULER.GAMMA
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=multisteps, gamma=gamma)

# LambdaLR 스케줄러 설정
lambda1 = lambda epoch: epoch / warmup_epochs if epoch < warmup_epochs else 1 # Warmup을 위한 Lambda 함수 정의
scheduler_warmup = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

# MultiStepLR 스케줄러 설정
scheduler_multistep = optim.lr_scheduler.MultiStepLR(optimizer, milestones=multisteps, gamma=gamma)

In [10]:
device = 'cuda:2'
model.to(device)

model_save = False
model_path = '../../models/swin2/simmim.pth'

In [11]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(train_epochs):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        image, mask = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            loss = model(image, mask)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        if simmim_config.TRAIN.CLIP_GRAD:
            clip_grad_norm_(model.parameters(), max_norm=simmim_config.TRAIN.CLIP_GRAD)
        else:
            clip_grad_norm_(model.parameters())

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        if epoch <= warmup_epochs:
            scheduler_warmup.step()
        else:
            scheduler_multistep.step()
        # scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    losses.append(epoch_loss)

    # 모델 저장
    if epoch_loss < best_loss:
        
        best_loss = epoch_loss
        vit_save = model_save
        if vit_save:
            torch.save(model.state_dict(), model_path)
        
    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss:.4f}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if vit_save:
        text += f' - model saved!'
        vit_save = False    
        
    print(text)


Epoch 1: 100%|██████████| 26/26 [00:23<00:00,  1.09it/s]

	Loss: 1.1252, LR: 0.0001, Duration: 24.44 sec



Epoch 2: 100%|██████████| 26/26 [00:22<00:00,  1.18it/s]

	Loss: 1.0885, LR: 0.0001, Duration: 22.79 sec



Epoch 3: 100%|██████████| 26/26 [00:21<00:00,  1.21it/s]

...

	Loss: 0.5656, LR: 1e-05, Duration: 22.57 sec



Epoch 99: 100%|██████████| 26/26 [00:21<00:00,  1.20it/s]

	Loss: 0.5651, LR: 1e-05, Duration: 22.56 sec



Epoch 100: 100%|██████████| 26/26 [00:21<00:00,  1.21it/s]

	Loss: 0.5660, LR: 1e-05, Duration: 22.34 sec


In [12]:
torch.save(model.state_dict(), model_path)

In [ ]:
device = 'cuda:2'

model_save = False
model_path = '../../models/swin2/simmim.pth'

In [15]:
swin = SwinTransformerV2(pretrained_window_sizes=[7,7,7,7], ape=True)
swin.load_state_dict(model.encoder.state_dict(), strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['mask_token'])

In [ ]:
swin = SwinTransformerV2(pretrained_window_sizes=[6,6,6,6], ape=True)

In [ ]:
swin_config = yaml.load(open('config/train.yaml'), Loader=yaml.FullLoader)
swin_config

In [ ]:
config = Box(swin_config)
model = swin

print(f"==============> Loading weight {config.MODEL.PRETRAINED} for fine-tuning......")
state_dict = torch.load(config.MODEL.PRETRAINED, map_location='cpu')

# remain encoder only
not_encoder_keys = [k for k in state_dict.keys() if 'encoder' not in k]
for k in not_encoder_keys:
    del state_dict[k]

# delete relative_position_index since we always re-init it
relative_position_index_keys = [k for k in state_dict.keys() if "relative_position_index" in k]
for k in relative_position_index_keys:
    del state_dict[k]

# delete relative_coords_table since we always re-init it
relative_position_index_keys = [k for k in state_dict.keys() if "relative_coords_table" in k]
for k in relative_position_index_keys:
    del state_dict[k]

# delete attn_mask since we always re-init it
attn_mask_keys = [k for k in state_dict.keys() if "attn_mask" in k]
for k in attn_mask_keys:
    del state_dict[k]

# bicubic interpolate relative_position_bias_table if not match
relative_position_bias_table_keys = [k for k in state_dict.keys() if "relative_position_bias_table" in k]
for k in relative_position_bias_table_keys:
    relative_position_bias_table_pretrained = state_dict[k]
    relative_position_bias_table_current = model.state_dict()[k]
    L1, nH1 = relative_position_bias_table_pretrained.size()
    L2, nH2 = relative_position_bias_table_current.size()
    if nH1 != nH2:
        print(f"Error in loading {k}, passing......")
    else:
        if L1 != L2:
            # bicubic interpolate relative_position_bias_table if not match
            S1 = int(L1 ** 0.5)
            S2 = int(L2 ** 0.5)
            relative_position_bias_table_pretrained_resized = torch.nn.functional.interpolate(
                relative_position_bias_table_pretrained.permute(1, 0).view(1, nH1, S1, S1), size=(S2, S2),
                mode='bicubic')
            state_dict[k] = relative_position_bias_table_pretrained_resized.view(nH2, L2).permute(1, 0)

# bicubic interpolate absolute_pos_embed if not match
absolute_pos_embed_keys = [k for k in state_dict.keys() if "absolute_pos_embed" in k]
for k in absolute_pos_embed_keys:
    # dpe
    absolute_pos_embed_pretrained = state_dict[k]
    absolute_pos_embed_current = model.state_dict()[k.replace('encoder.','')]
    _, L1, C1 = absolute_pos_embed_pretrained.size()
    _, L2, C2 = absolute_pos_embed_current.size()
    if C1 != C1:
        print(f"Error in loading {k}, passing......")
    else:
        if L1 != L2:
            S1 = int(L1 ** 0.5)
            S2 = int(L2 ** 0.5)
            absolute_pos_embed_pretrained = absolute_pos_embed_pretrained.reshape(-1, S1, S1, C1)
            absolute_pos_embed_pretrained = absolute_pos_embed_pretrained.permute(0, 3, 1, 2)
            absolute_pos_embed_pretrained_resized = torch.nn.functional.interpolate(
                absolute_pos_embed_pretrained, size=(S2, S2), mode='bicubic')
            absolute_pos_embed_pretrained_resized = absolute_pos_embed_pretrained_resized.permute(0, 2, 3, 1)
            absolute_pos_embed_pretrained_resized = absolute_pos_embed_pretrained_resized.flatten(1, 2)
            state_dict[k] = absolute_pos_embed_pretrained_resized

# check classifier, if not match, then re-init classifier to zero
head_bias_pretrained = state_dict['encoder.classifier.bias']
Nc1 = head_bias_pretrained.shape[0]
Nc2 = model.classifier.bias.shape[0]
if (Nc1 != Nc2):
    torch.nn.init.constant_(model.classifier.bias, 0.)
    torch.nn.init.constant_(model.classifier.weight, 0.)
    del state_dict['encoder.classifier.weight']
    del state_dict['encoder.classifier.bias']
    print(f"Error in loading classifier head, re-init classifier head to 0")

msg = model.load_state_dict(state_dict, strict=False)
print(msg)

print(f"=> loaded successfully '{config.MODEL.PRETRAINED}'")

torch.cuda.empty_cache()

In [ ]:
def load_pretrained(config, model):
    print(f"==============> Loading weight {config.MODEL.PRETRAINED} for fine-tuning......")
    state_dict = torch.load(config.MODEL.PRETRAINED, map_location='cpu')

    # remain encoder only
    not_encoder_keys = [k for k in state_dict.keys() if 'encoder' not in k]
    for k in not_encoder_keys:
        del state_dict[k]

    # delete relative_position_index since we always re-init it
    relative_position_index_keys = [k for k in state_dict.keys() if "relative_position_index" in k]
    for k in relative_position_index_keys:
        del state_dict[k]

    # delete relative_coords_table since we always re-init it
    relative_position_index_keys = [k for k in state_dict.keys() if "relative_coords_table" in k]
    for k in relative_position_index_keys:
        del state_dict[k]

    # delete attn_mask since we always re-init it
    attn_mask_keys = [k for k in state_dict.keys() if "attn_mask" in k]
    for k in attn_mask_keys:
        del state_dict[k]

    # bicubic interpolate relative_position_bias_table if not match
    relative_position_bias_table_keys = [k for k in state_dict.keys() if "relative_position_bias_table" in k]
    for k in relative_position_bias_table_keys:
        relative_position_bias_table_pretrained = state_dict[k]
        relative_position_bias_table_current = model.state_dict()[k]
        L1, nH1 = relative_position_bias_table_pretrained.size()
        L2, nH2 = relative_position_bias_table_current.size()
        if nH1 != nH2:
            print(f"Error in loading {k}, passing......")
        else:
            if L1 != L2:
                # bicubic interpolate relative_position_bias_table if not match
                S1 = int(L1 ** 0.5)
                S2 = int(L2 ** 0.5)
                relative_position_bias_table_pretrained_resized = torch.nn.functional.interpolate(
                    relative_position_bias_table_pretrained.permute(1, 0).view(1, nH1, S1, S1), size=(S2, S2),
                    mode='bicubic')
                state_dict[k] = relative_position_bias_table_pretrained_resized.view(nH2, L2).permute(1, 0)

    # bicubic interpolate absolute_pos_embed if not match
    absolute_pos_embed_keys = [k for k in state_dict.keys() if "absolute_pos_embed" in k]
    for k in absolute_pos_embed_keys:
        # dpe
        absolute_pos_embed_pretrained = state_dict[k]
        absolute_pos_embed_current = model.state_dict()[k.replace('encoder.','')]
        _, L1, C1 = absolute_pos_embed_pretrained.size()
        _, L2, C2 = absolute_pos_embed_current.size()
        if C1 != C1:
            print(f"Error in loading {k}, passing......")
        else:
            if L1 != L2:
                S1 = int(L1 ** 0.5)
                S2 = int(L2 ** 0.5)
                absolute_pos_embed_pretrained = absolute_pos_embed_pretrained.reshape(-1, S1, S1, C1)
                absolute_pos_embed_pretrained = absolute_pos_embed_pretrained.permute(0, 3, 1, 2)
                absolute_pos_embed_pretrained_resized = torch.nn.functional.interpolate(
                    absolute_pos_embed_pretrained, size=(S2, S2), mode='bicubic')
                absolute_pos_embed_pretrained_resized = absolute_pos_embed_pretrained_resized.permute(0, 2, 3, 1)
                absolute_pos_embed_pretrained_resized = absolute_pos_embed_pretrained_resized.flatten(1, 2)
                state_dict[k] = absolute_pos_embed_pretrained_resized

    # check classifier, if not match, then re-init classifier to zero
    head_bias_pretrained = state_dict['encoder.classifier.bias']
    Nc1 = head_bias_pretrained.shape[0]
    Nc2 = model.classifier.bias.shape[0]
    if (Nc1 != Nc2):
        torch.nn.init.constant_(model.classifier.bias, 0.)
        torch.nn.init.constant_(model.classifier.weight, 0.)
        del state_dict['encoder.classifier.weight']
        del state_dict['encoder.classifier.bias']
        print(f"Error in loading classifier head, re-init classifier head to 0")

    msg = model.load_state_dict(state_dict, strict=False)
    print(msg)

    print(f"=> loaded successfully '{config.MODEL.PRETRAINED}'")

    torch.cuda.empty_cache()

In [ ]:
swin_config = Box(swin_config)
load_pretrained(swin_config, swin)

In [ ]:
swin.load_state_dict(model.encoder.state_dict(), strict=False)

In [16]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [17]:
# Transforms 정의하기
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8,1), interpolation=transforms.InterpolationMode.LANCZOS),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.9, scale=(0.02, 0.33)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_dir = '../../data/sports'
batch_size = 960

train_path = data_dir+'/train'
valid_path = data_dir+'/valid'
test_path = data_dir+'/test'

# dataset load
train_data = ImageFolder(train_path, transform=train_transform)
valid_data = ImageFolder(valid_path, transform=test_transform)
test_data = ImageFolder(test_path, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [18]:
max_norm = 5.0

swin.to(device)

SwinTransformerV2(
  (embeddings): embeddings(
    (patch_embeddings): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (stages): ModuleList(
    (0): StageLayer(
      (blocks): ModuleList(
        (0-1): 2 x SwinTransformerBlock(
          (attn): WindowAttention(
            (crpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=384, bias=True)
              (1): ReLU(inplace=True)
              (2): Dropout(p=0.125, inplace=False)
              (3): Linear(in_features=384, out_features=3, bias=False)
            )
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
  

In [19]:
mixup_fn = Mixup(mixup_alpha=1., 
                cutmix_alpha=1., 
                prob=1., 
                switch_prob=0.5, 
                mode='batch',
                label_smoothing=.1,
                num_classes=100)

epochs = 100

criterion = nn.CrossEntropyLoss(label_smoothing=0.)
optimizer = optim.AdamW(swin.parameters(), lr=1e-3, weight_decay=5e-3)
warmup_steps = int(len(train_loader)*epochs*0.1)
train_steps = len(train_loader)*epochs
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, 
                                                        num_warmup_steps=warmup_steps, 
                                                        num_training_steps=train_steps,
                                                        num_cycles=0.5)

2024-01-10 15:48:00.880244: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-10 15:48:00.880312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-10 15:48:00.881078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-10 15:48:00.886287: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 15:48:01.667195: W tensorflow/compiler/tf2

In [20]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    swin.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = swin(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(swin.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    swin.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = swin(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        # vit_save = True
        # if vit_save:
        #     torch.save(swin.state_dict(), )

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    # if vit_save:
    #     text += f' - swin saved!'
    #     vit_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

Epoch 1: 100%|██████████| 15/15 [00:51<00:00,  3.44s/it]


	Loss: 4.5992762565612795, Val Loss: 4.361097812652588, LR: 0.0001, Duration: 52.77 sec


Epoch 2: 100%|██████████| 15/15 [00:50<00:00,  3.37s/it]


	Loss: 4.420850372314453, Val Loss: 4.1072845458984375, LR: 0.0002, Duration: 51.46 sec


Epoch 3: 100%|██████████| 15/15 [00:50<00:00,  3.34s/it]


...

	Loss: 2.9519651254018147, Val Loss: 1.0567539930343628, LR: 1.2179748700879012e-06, Duration: 50.76 sec


Epoch 99: 100%|██████████| 15/15 [00:49<00:00,  3.33s/it]


	Loss: 2.5671448548634848, Val Loss: 1.0566096305847168, LR: 3.0458649045211895e-07, Duration: 50.84 sec


Epoch 100: 100%|██████████| 15/15 [00:49<00:00,  3.32s/it]


	Loss: 2.682647705078125, Val Loss: 1.0565671920776367, LR: 0.0, Duration: 50.70 sec
Epoch 당 평균 소요시간 : 50.90초


In [21]:
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [23]:
# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = swin(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

,Metric,Value
0,Accuracy,0.794000
1,Precision,0.827087
2,Recall,0.794000
3,F1 Score,0.784236
